In [ ]:
import json
import geopandas as gpd
import matplotlib.pyplot as plt
import geopy.distance
import operator

with open('spb.geojson') as f:
    spb = json.load(f)
spb_pos = (30.3351,59.9343)

In [ ]:
#amount of elements of different types
print(spb.keys())
print(spb['type'])
print(len(spb['features']))

features = spb['features'][:]
types = {}
for feature in features:
    if feature['geometry']['type'] not in types:
        types[feature['geometry']['type']] = 0
    else:
        types[feature['geometry']['type']] = types[feature['geometry']['type']] + 1

print(types)

In [ ]:
def sorted_features():
    # amounts of properties sorted in reverse order
    properties = {}
    features = spb['features'][:]

    for feature in features:
        for prop in feature['properties']: 
            if prop not in properties:
                properties[prop] = 0
            else:
                properties[prop] = properties[prop] + 1


    sorted_props = sorted(properties.items(), key=operator.itemgetter(1), reverse=True)

    #prints properties from most used to least
    for el in sorted_props:
        print(el[0] + ' - ' + str(el[1]))

def get_feature_dist(pos, feature):
    if feature['geometry']['type'] == 'Point':
        return  geopy.distance.distance(pos, feature['geometry']['coordinates']).m
    
    elif feature['geometry']['type'] == 'LineString':
        min_dist = float("inf")
        for my_pos in feature['geometry']['coordinates']:
            dist = geopy.distance.distance(pos,my_pos).m
            if dist < min_dist:
                min_dist = dist
        return min_dist
    
    else:
#         return -1# multi polygon
        min_dist = float("inf")
        for l1 in feature['geometry']['coordinates']:
            for l2 in l1:
                for my_pos in l2:
                    dist = geopy.distance.distance(pos,my_pos).m
                    if dist < min_dist:
                        min_dist = dist
        return min_dist
        
        
def leisures_and_amenities():
    
    leisure_features = []
    features = spb['features'][:]
    leisures = {}
    amenities = {}
    
    for feature in features:
        
        props = feature['properties']
        if 'leisure' in props:
            leisure_features.append(feature)
            if props['leisure'] in leisures:
                leisures[props['leisure']] += 1
            else:
                leisures[props['leisure']] = 1
                
        if 'amenity' in props:
            leisure_features.append(feature)
            if props['amenity'] in amenities:
                amenities[props['amenity']] += 1
            else:
                amenities[props['amenity']] = 1

    sorted_leisures = sorted(leisures.items(), key=operator.itemgetter(1), reverse=True)
    sorted_amenities = sorted(amenities.items(), key=operator.itemgetter(1), reverse=True)

    leisures = []
    #prints properties from most used to least
    for el in sorted_leisures:
        leisures.append(el[0])
        print(el[0] + ' - ' + str(el[1]))
        
    print()
    for el in sorted_amenities:
        leisures.append(el[0])
        print(el[0] + ' - ' + str(el[1]))
        
    return leisures
    

def get_close_leisures(pos):

    close_objects = {}
    closer_objects = {}
    distance_to_closest = {}
    leisures = leisures_and_amenities()
    long_distance_threshold = 1000 # in meters
    close_distance_threshold = long_distance_threshold / 2
    
    for feature in spb['features']:
        #my_pos = get_feature_dist(pos, feature)
        #print(my_pos)
        #dist = geopy.distance.distance(spb_pos,my_pos).m
        dist = get_feature_dist(pos, feature)
        if dist == -1:
            print('found multipolygon: ')
            break
        # if close enough
        if dist < long_distance_threshold:
        
            for prop in feature['properties']:
                if prop in leisures:
                    if prop not in distance_to_closest:
                        distance_to_closest[prop] = dist
                    else:
                        distance_to_closest[prop] = min(dist, distance_to_closest[prop])
                    
                    if prop not in close_objects:
                        close_objects[prop] = [(feature, dist)]
                    else:
                        close_objects[prop].append((feature, dist))
                    print(feature['properties'])
                    #close_objects.append(feature)
                    break
                    
            if dist < close_distance_threshold:
                for prop in feature['properties']:
                    if prop in leisures:
                        if prop not in closer_objects:
                            closer_objects[prop] = [(feature, dist)]
                        else:
                            closer_objects[prop].append((feature, dist))
                        print(feature['properties'])
                        #close_objects.append(feature)
                        break
    return close_objects, closer_objects, distance_to_closest



In [ ]:
distance

In [ ]:

for el in closer:
    print(el,len(close[el]))
#     for feature in close[el]:
#         print(feature[0]['properties'])
#         print(feature[1])

In [ ]:

print('in 1000m radius')
for el in close:
    print(el,len(close[el]))
    
print()
print('in 500m raduis')

for el in closer:
    print(el,len(closer[el]))
    
print()
print('distances to closest')
print(distance)

In [ ]:
spb_pos = (30.3351,59.9343)
close, closer, distance = get_close_leisures(spb_pos)